In [16]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split,GridSearchCV,cross_val_score
from sklearn.metrics import accuracy_score, classification_report,roc_curve, confusion_matrix
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
import warnings
warnings.filterwarnings("ignore")

---
# Import and Preprocess Yahoo S&P 500 Health Care Sector Data

In [17]:
import pandas as pd
import numpy as np
import yfinance as yf
from collections import deque 

In [46]:
SPHC = yf.download('^SP500-35', start='2019-09-05', end='2022-06-04')
SPHC.to_csv('SPHC.csv')

[*********************100%***********************]  1 of 1 completed


In [47]:
SPHC.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-09-05,1047.000000,1052.969971,1043.900024,1050.459961,1050.459961,151221300
2019-09-06,1052.619995,1056.599976,1051.689941,1053.739990,1053.739990,156235900
2019-09-09,1053.819946,1054.609985,1040.189941,1044.069946,1044.069946,185006400
2019-09-10,1041.969971,1047.119995,1027.660034,1047.119995,1047.119995,230308700
2019-09-11,1047.939941,1057.709961,1046.369995,1057.709961,1057.709961,168617800


In [48]:
SPHC.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 693 entries, 2019-09-05 to 2022-06-03
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       693 non-null    float64
 1   High       693 non-null    float64
 2   Low        693 non-null    float64
 3   Close      693 non-null    float64
 4   Adj Close  693 non-null    float64
 5   Volume     693 non-null    int64  
dtypes: float64(5), int64(1)
memory usage: 37.9 KB


In [49]:
# Preprocess Data

# Store Date and Change Index
SPHC["Date"] = SPHC.index
index = pd.Index(range(0, len(SPHC)))
SPHC = SPHC.set_index(index)

# Diff
SPHC['Diff'] = SPHC['High'] - SPHC['Low']

# Diff20
Diff20 = [None for i in range(19)]
Diff20sum = 0

# DiffChange
DiffChange = [None for i in range(20)]

# Return
Return = [None]

# Return5
Return5 = [None for i in range(5)]

# MA10
MA10 = [None for i in range(9)]
MA10sum = 0

# MA30
MA30 = [None for i in range(29)]
MA30sum = 0

# MA100
MA100 = [None for i in range(99)]
MA100sum = 0

# MA10-30
MA1030 = [None for i in range(29)]

# MA30-100
MA30100 = [None for i in range(99)]

# High30
High30 = [None for i in range(29)]
High30q = deque()

# Low30
Low30 = [None for i in range(29)]
Low30q = deque()

# Break30
Break30 = [None for i in range(30)]

# Vol20
Vol20 = [None for i in range(20)]
Vol20q = deque()

# Label
Label = [None for i in range(99)]

for i in range(len(SPHC)):
    curr = SPHC.loc[i]["Adj Close"]
    
    # Diff20
    Diff20sum+=SPHC.loc[i]['Diff']
    if i>19:
        Diff20sum-=SPHC.loc[i-20]['Diff']
    if i>18:
        Diff20.append(Diff20sum/20)
    
    # DiffChange:
    if i>19:
        if SPHC.loc[i]['Diff']>Diff20[i-1]:
            DiffChange.append(1)
        elif SPHC.loc[i]['Diff']==Diff20[i-1]:
            DiffChange.append(0)
        else:
            DiffChange.append(-1)
    
    # Return
    if i>0:
        prev = SPHC.loc[i-1]["Adj Close"]
        Return.append((curr - prev)/prev*100)
    
    # Return5
    if i>4:
        prev5 = SPHC.loc[i-5]["Adj Close"]
        Return5.append((curr - prev5)/prev5*100)
    
    # MA10
    MA10sum+=curr
    if i>9:
        MA10sum-=SPHC.loc[i-10]['Adj Close']
    if i>8:
        MA10.append(MA10sum/10)
    
    # MA30
    MA30sum+=curr
    if i>29:
        MA30sum-=SPHC.loc[i-30]['Adj Close']
    if i>28:
        MA30.append(MA30sum/30)
    
    # MA100
    MA100sum+=curr
    if i>99:
        MA100sum-=SPHC.loc[i-100]['Adj Close']
    if i>98:
        MA100.append(MA100sum/100)
    
    # MA10-30
    if i>28:
        if MA10[i]>MA30[i]:
            MA1030.append(1)
        elif MA10[i]==MA30[i]:
            MA1030.append(0)
        else:
            MA1030.append(-1)
    
    # MA30-100
    if i>98:
        if MA30[i]>MA100[i]:
            MA30100.append(1)
        elif MA30[i]==MA100[i]:
            MA30100.append(0)
        else:
            MA30100.append(-1)
    
    # High30
    while High30q and curr>High30q[-1]:
        High30q.pop()
    High30q.append(curr)
    if i>29 and High30q[0] == SPHC.loc[i-30]['Adj Close']:
        High30q.popleft()
    if i>28:
        High30.append(High30q[0])
    
    # Low30
    while Low30q and curr<Low30q[-1]:
        Low30q.pop()
    Low30q.append(curr)
    if i>29 and Low30q[0] == SPHC.loc[i-30]['Adj Close']:
        Low30q.popleft()
    if i>28:
        Low30.append(Low30q[0])
    
    # Break 30
    if i>=30:
        if curr>High30[i-1]:
            Break30.append(1)
        elif curr<Low30[i-1]:
            Break30.append(-1)
        else:
            Break30.append(0)
    
    # Vol20
    if i>0:
        Vol20q.append(Return[i])
    if len(Vol20q)>20:
        Vol20q.popleft()
    if i>19:
        Vol20.append(np.std(Vol20q))

# Label
for i in range(99, len(SPHC)-1):
    if Return[i+1]>0:
        Label.append(1)
    elif Return[i+1]==0:
        Label.append(0)
    else:
        Label.append(-1)
Label.append(None)
            
        
SPHC["Diff20"] = Diff20
SPHC["DiffChange"] = DiffChange
SPHC["Return"] = Return
SPHC["Return5"] = Return5
SPHC["MA10"] = MA10
SPHC["MA30"] = MA30
SPHC["MA100"] = MA100
SPHC["MA1030"] = MA1030
SPHC["MA30100"] = MA30100
SPHC["High30"] = High30
SPHC["Low30"] = Low30
SPHC["Break30"] = Break30
SPHC["Vol20"] = Vol20
SPHC["Label"] = Label

In [50]:
SPHC.head(11)

,Open,High,Low,Close,Adj Close,Volume,Date,Diff,Diff20,DiffChange,...,MA10,MA30,MA100,MA1030,MA30100,High30,Low30,Break30,Vol20,Label
0,1047.000000,1052.969971,1043.900024,1050.459961,1050.459961,151221300,2019-09-05,9.069946,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1052.619995,1056.599976,1051.689941,1053.739990,1053.739990,156235900,2019-09-06,4.910034,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1053.819946,1054.609985,1040.189941,1044.069946,1044.069946,185006400,2019-09-09,14.420044,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1041.969971,1047.119995,1027.660034,1047.119995,1047.119995,230308700,2019-09-10,19.459961,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1047.939941,1057.709961,1046.369995,1057.709961,1057.709961,168617800,2019-09-11,11.339966,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1061.989990,1066.099976,1056.270020,1056.439941,1056.439941,169735100,2019-09-12,9.829956,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1057.949951,1063.420044,1053.280029,1055.709961,1055.709961,162282400,2019-09-13,10.140015,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1052.489990,1055.689941,1048.670044,1053.260010,1053.260010,128026100,2019-09-16,7.019897,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1052.589966,1057.540039,1051.569946,1054.780029,1054.780029,149497800,2019-09-17,5.970093,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1054.150024,1055.910034,1047.130005,1054.839966,1054.839966,137667100,2019-09-18,8.780029,NaN,NaN,...,1052.812976,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
# Drop None Rows
SPHC.drop([len(SPHC)-1], inplace = True)
SPHC.drop([i for i in range(99)], inplace = True)

# Select features
SPHC = SPHC[["Date","High","Low","Volume","Diff","DiffChange","Return","Return5","MA1030","MA30100","Break30","Vol20","Label"]]

# Reset Index
index = pd.Index(range(0, len(SPHC)))
SPHC = SPHC.set_index(index)
SPHC = SPHC.astype({'Label':'int'})

In [52]:
SPHC

,Date,High,Low,Volume,Diff,DiffChange,Return,Return5,MA1030,MA30100,Break30,Vol20,Label
0,2020-01-28,1194.689941,1184.349976,202157900,10.339966,1.0,0.477113,-2.326380,1.0,1.0,0.0,0.617034,-1
1,2020-01-29,1195.069946,1186.439941,166189700,8.630005,-1.0,-0.279020,-2.713914,1.0,1.0,0.0,0.602541,-1
2,2020-01-30,1179.380005,1167.339966,199618700,12.040039,1.0,-0.775370,-2.987806,1.0,1.0,0.0,0.621944,-1
3,2020-01-31,1173.199951,1150.369995,228459400,22.829956,1.0,-1.985833,-3.292691,1.0,1.0,-1.0,0.748871,1
4,2020-02-03,1172.630005,1161.109985,207902400,11.520020,1.0,0.925513,-1.652574,-1.0,1.0,0.0,0.765988,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
588,2022-05-26,1538.979980,1526.260010,178529300,12.719971,-1.0,0.415281,2.741086,-1.0,-1.0,0.0,1.343798,1
589,2022-05-27,1559.339966,1529.569946,185013000,29.770020,-1.0,1.717533,3.205348,-1.0,-1.0,0.0,1.277321,-1
590,2022-05-31,1559.339966,1525.959961,362578700,33.380005,1.0,-1.373653,0.975666,-1.0,-1.0,0.0,1.308182,-1
591,2022-06-01,1543.839966,1504.239990,159014200,39.599976,1.0,-1.423352,-0.714495,-1.0,-1.0,0.0,1.348318,1


---
# Machine Learning Models

## Split the Data into the Train and Test set

In [10]:
data = SPHC
list_features = list(data)
list_features.remove('Date')
list_features.remove('Label')
x = data[list_features]
y = data['Label']
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.30, random_state=7)

## Improve the Decision Tree Model

In [11]:
#Improve the Decision Tree Model
#Using grid search, the parameters of max_depth and min_samples_split are adjusted in a small range
param_grid = {
    'max_depth':np.arange(2, 6),
    'min_samples_split':np.arange(2, 18)}

rfc = DecisionTreeClassifier(random_state=10)
GS = GridSearchCV(rfc,param_grid,cv=10)
GS.fit(x_train,y_train)
dt = [GS.best_params_['max_depth'],GS.best_params_['min_samples_split']]
print(GS.best_params_)
print(GS.best_score_)

{'max_depth': 5, 'min_samples_split': 10}
0.48512605042016804


## Improve the Random Forest Model

In [12]:
#Improve the Random Forest  Model
#Using grid search, the parameters of max_depth and min_samples_split are adjusted in a small range
param_grid = {
    'max_depth':np.arange(2, 6),
    'min_samples_split':np.arange(2, 10)}

rfc = RandomForestClassifier(random_state=10)
GS = GridSearchCV(rfc,param_grid,cv=10)
GS.fit(x_train,y_train)
rf = [GS.best_params_['max_depth'],GS.best_params_['min_samples_split']]
print(GS.best_params_)
print(GS.best_score_)

{'max_depth': 2, 'min_samples_split': 6}
0.5178151260504201


## Improve the Logistic Regression Model

In [13]:
#Improve the Logistic Regression Model
#Using grid search, the parameter of max_iter is adjusted in a small range
param_grid = {
    'max_iter':np.arange(300, 2000,100)}

rfc = LogisticRegression(solver='lbfgs')
GS = GridSearchCV(rfc,param_grid,cv=10)
GS.fit(x_train,y_train)
lg = GS.best_params_['max_iter']
print(GS.best_params_)
print(GS.best_score_)

{'max_iter': 300}
0.49420168067226894


## Improve the LightGBM Model

In [14]:
#Improve the Decision Tree Model
#Using grid search, the parameters of max_depth and min_samples_split are adjusted in a small range
param_grid = {
    'max_depth':np.arange(2, 6),
    'num_leaves':np.arange(2, 18)}

rfc = LGBMClassifier(random_state=10)
GS = GridSearchCV(rfc,param_grid,cv=10)
GS.fit(x_train,y_train)
lgb = [GS.best_params_['max_depth'],GS.best_params_['num_leaves']]
print(GS.best_params_)
print(GS.best_score_)

KeyboardInterrupt: 

## Compare Models

In [ ]:
#We implement a class of methods to call all previous steps into one unique cell.
class classi:
    def __init__(self,estimators):
        self.algo=estimators
        self.best_estimateurs={}
     #Fitting the models before the prediction phase
    
    #Prediction used on the test set for all our models
    def pred(self,x_train,y_train,x_test,y_test):
        pred={}
        for estim in self.algo:
            self.algo[estim]['mod'].fit(x_train,y_train)
            pred[estim]=self.algo[estim]['mod'].predict(x_test)
        return pred
    
    def roc_curve(self,pred):
        plt.figure(figsize=(10,5))
        for estim in pred.keys():
            fpr,tpr,threshold = roc_curve(y_test,pred[estim])
            sns.set_style('whitegrid')
            plt.title('Receiver Operating Characteristic Curve')
            plt.plot(fpr,tpr,label=f'{estim}')
        plt.ylabel('True positive rate')
        plt.xlabel('False positive rate')
        plt.legend()
        plt.show()
    
    def show_score(self,y_test,pred):
        conf_mat={}
        acc_score={}
        for estim in self.algo:
            print(f"--------------{estim}---------------------------------------------------------------------")
            conf_mat[estim]=confusion_matrix(y_test,pred[estim])
            acc_score[estim]=accuracy_score(y_test,pred[estim])
            print(f'Using the confusion matrix, we have : \n {conf_mat[estim]}')
            print('\n')
            print(f'The accuracy score of {estim} is:\n {acc_score[estim]*100}')
            print('\n')
            print('We present the classification report with all the metrics related to the predictive score')
            print(classification_report(y_test,pred[estim]))
            print(f"*"*40)
            print("\n")
        return acc_score

    def model_eval(self,acc_score):
        accuracy=acc_score.copy()
        accuracy.update((x,y*100) for x,y in acc_score.items())
        models=list(acc_score.keys())
        scores=list(accuracy.values())
        mod_ev=pd.DataFrame({'Model':models,'Accuracy':scores})
        mod_ev
        return mod_ev

In [ ]:
# Estimators to be called 
estimators={"Decision Trees":{"mod":DecisionTreeClassifier(max_depth= dt[0], min_samples_split= dt[1], random_state=10),"param":0},
           "Random Forest":{'mod':RandomForestClassifier(max_depth=rf[0], min_samples_split=rf[1], random_state=10),"param":0},
           'Naive Bayes':{"mod":GaussianNB(),"param":0},
           'Logistic Regression':{"mod": LogisticRegression(solver='lbfgs', max_iter=lg),"param":0},
           "LightGBM":{"mod": LGBMClassifier(max_depth= lgb[0], num_leaves= lgb[1], random_state=10),"param":0}}

In [ ]:
classification= classi(estimators)

In [ ]:
pred=classification.pred(x_train,y_train,x_test,y_test)
classification.roc_curve(pred)

In [ ]:
acc_score=classification.show_score(y_test,pred)
model_ev=classification.model_eval(acc_score)
model_ev